# System

In [ ]:
if is_debug: print("\n---------- This notenook name: nb_bronze_function_extract_csv - Start ----------\n")

In [ ]:
medallion_name = "Bronze"
if is_debug: print(f"medallion_name: {medallion_name}")

In [ ]:
# Current session info
if is_debug: display(ss.getActiveSession())

# Function

## Read CSV file

In [ ]:
if "CSV" in list_technology:
    def fn_read_csv_file(dict_parameter):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            if dict_parameter["does_csv_file_exist"] == True:
                # Replace placeholders r, n, rn with \r, \n, \r\n
                match dict_parameter["row_separator"]:
                    case 'r':  row_separator = "\r"
                    case 'n':  row_separator = "\n"
                    case 'rn': row_separator = "\r\n"
                    case _:    dict_parameter["row_separator"]
                
                options = {"header": dict_parameter["has_header"]
, "delimiter": dict_parameter["delimiter"]
, "lineSep": row_separator
, "quote": dict_parameter["quote"]
, "ignoreLeadingWhiteSpace": "True"}

                # Read CSV file
                sdf = spark.read \
                    .options(**options) \
                    .csv(dict_parameter["csv_file_path_clean"])
                sdf_count = sdf.count()

                # Apply the data types from "global_parameter.projet_documentation_data_file_name"
                sdf_col    = fn_get_extract_column(dict_parameter)[2]
                sdf_col    = sdf_col.withColumn("alias_name", fn_clean_accent_in_sdf_column("column_name")[2])
                sdf_col    = fn_replace_all_not_alphanumericals_in_single_column_in_sdf(sdf_col, "alias_name", "_")[2]
                select_col = fn_get_select_column("sdf", sdf_col)[2]                
                sdf        = eval(str(select_col))
            else:
                alert             = "Warning"
                alert_description = f"CSV file \"{csv_file_path}\" does not exist."

            alert             = "Success"
            alert_description = f"CSV file (rows): {sdf_count}"

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, sdf, sdf_count)                
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

# Operation

## Extract CSV

In [ ]:
if "CSV" in list_technology:
    def fn_extract_csv(
        technology
        , frequency
    ):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            # Rename the existing files in lh_bronze
            sdf_renamed = fn_rename_files_in_lh_bronze(technology, frequency, "Archive")[1]

            # Get extract table list
            dict_parameter_table_list     = {"technology": technology, "frequency": frequency, "action": "Extract"}
            rv_extract_table_list         = fn_get_extract_table_list(dict_parameter_table_list)
            sdf_extract_table_list_status = rv_extract_table_list[0]
            sdf_extract_table_list        = rv_extract_table_list[2]
            sdf_extract_table_list_count  = rv_extract_table_list[3]

            if is_debug:
                print(f"sdf_extract_table_list_status: {sdf_extract_table_list_status}")
                print(f"sdf_extract_table_list:")
                display(sdf_extract_table_list)
                print(f"sdf_extract_table_list_count:  {sdf_extract_table_list_count}")

            # Loop extract table list
            if sdf_extract_table_list_count > 0:
                # etl_r --> sdf_extract_table_list.row
                for etl_r in sdf_extract_table_list \
                    .sort(sdf_extract_table_list.technology.asc(),
                        sdf_extract_table_list.frequency.asc(),                        
                        sdf_extract_table_list.folder_name.asc(),
                        sdf_extract_table_list.file_name.asc()
                    ).collect():

                    dict_parameter = {"process_timestamp": global_parameter.process_timestamp
, "technology": etl_r.technology
, "frequency": etl_r.frequency
, "folder_name": etl_r.folder_name
, "file_name": etl_r.file_name
, "has_header": etl_r.has_header
, "delimiter": etl_r.delimiter
, "row_separator": etl_r.row_separator
, "quote": etl_r.quote}

                    if is_debug:
                        print(f"sdf_extract_table_list.etl_r:")
                        print(f"  technology:    {etl_r.technology}")
                        print(f"  frequency:     {etl_r.frequency}")
                        print(f"  folder_name:   {etl_r.folder_name}")
                        print(f"  file_name:     {etl_r.file_name}")                    
                        print(f"  has_header:    {etl_r.has_header}")
                        print(f"  delimiter:     {etl_r.delimiter}")
                        print(f"  row_separator: {etl_r.row_separator}")
                        print(f"  quote:         {etl_r.quote}")
                        print(f"dict_parameter: {dict_parameter}")
                    
                    # Clean up "technology", "folder_name", "file_name" - Start
                    # Clean accents and not alphanumerics
                    technology_clean  = fn_clean_not_alphanumeric_in_string(fn_clean_accent_in_string(technology)[2], "_")[2]
                    folder_name_clean = etl_r.folder_name.replace("/", "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789") # keep "/" while cleaning
                    folder_name_clean = fn_clean_not_alphanumeric_in_string(fn_clean_accent_in_string(folder_name_clean)[2], "_")[2]
                    folder_name_clean = folder_name_clean.replace("ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789", "/")
                    file_name_clean   = fn_clean_not_alphanumeric_in_string(fn_clean_accent_in_string(etl_r.file_name)[2], "_")[2]

                    dict_parameter["technology_clean"]  = technology_clean
                    dict_parameter["folder_name_clean"] = folder_name_clean
                    dict_parameter["file_name_clean"]   = file_name_clean

                    if is_debug:
                        print(f"folder_name_clean: >{folder_name_clean}<")
                        print(f"file_name_clean: >{file_name_clean}<")
                        print(f"dict_parameter: {dict_parameter}")
                    # Clean up "technology", "folder_name", "file_name" - End

                    # Generate source file path and check if file exists - Start
                    csv_file_path_clean = f"{global_parameter.abfs_path_lh_bronze}/Files/incoming/{etl_r.technology}/{folder_name_clean}/{file_name_clean}.csv"
                    csv_file_path_clean = csv_file_path_clean.replace("abfss://", "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789")
                    csv_file_path_clean = csv_file_path_clean.replace("//", "/")
                    csv_file_path_clean = csv_file_path_clean.replace("ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789", "abfss://")

                    does_csv_file_exist = fn_does_file_exist(csv_file_path_clean, dict_parameter)[2]

                    dict_parameter["csv_file_path_clean"] = csv_file_path_clean
                    dict_parameter["does_csv_file_exist"] = does_csv_file_exist

                    if is_debug:
                        print(f"csv_file_path_clean: {csv_file_path_clean}")
                        print(f"does_csv_file_exist: {does_csv_file_exist}")
                        print(f"dict_parameter: {dict_parameter}")
                    # Generate source file path and check if file exists - End
                    
                    if does_csv_file_exist == True:
                        # Extract file - Start
                        rv_csv_file         = fn_read_csv_file(dict_parameter)
                        sdf_csv_file_status = rv_csv_file[0]
                        sdf_csv_file        = rv_csv_file[2]
                        sdf_csv_file_count  = rv_csv_file[3]

                        if is_debug:
                            print(f"rv_csv_file:         {rv_csv_file}")
                            print(f"sdf_csv_file_status: {sdf_csv_file_status}")
                            print(f"sdf_csv_file (schema):")
                            sdf_csv_file.printSchema()
                            print(f"sdf_csv_file:")
                            display(sdf_csv_file)
                            print(f"sdf_csv_file_count:  {sdf_csv_file_count}")
                        # Extract file - End

                        # Save CSV file as table in "lh_bronze" - Start
                        # Replace "/" with "_" in "folder_name"
                        folder_name_in_table_name = folder_name_clean.replace("/", "_")

                        # Generate full table path
                        csv_file_table_path = f"{global_parameter.abfs_path_lh_bronze}/Tables/{technology_clean}_{folder_name_in_table_name}_{file_name_clean}"

                        # Remove double underscore (folder_name is empty)
                        csv_file_table_path = csv_file_table_path.replace(f"__", "_")

                        if is_debug: print(f"sdf_csv_file_table_path: {csv_file_table_path}")

                        fn_save_sdf_as_table(
                            sdf_csv_file
                            , "delta"
                            , "overwrite"
                            , csv_file_table_path
                            , dict_parameter
                        )
                        # Save CSV file as table in "lh_bronze" - End

                        # Copy and delete CSV file to "lh_bronze/Processed/CSV" (archive) - Start
                        now_year  = fn_get_now("year")[1]
                        now_month = fn_get_now("month")[1]
                        dstn_file_path = f"{global_parameter.abfs_path_lh_bronze}/Files/processed/{technology_clean}/\
{folder_name_clean}/{file_name_clean}/\
{now_year}/{now_month}/\
{file_name_clean}_{global_parameter.process_timestamp}.csv"

                        dstn_file_path = dstn_file_path.replace("abfss://", "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789")
                        dstn_file_path = dstn_file_path.replace("//", "/")
                        dstn_file_path = dstn_file_path.replace("ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789", "abfss://")

                        copy_file_status = fn_copy_file(csv_file_path_clean, dstn_file_path, dict_parameter)[0]
                        if copy_file_status == "Success": fn_delete_file(csv_file_path_clean, dict_parameter)

                        if is_debug:
                            print(f"csv_file_path_clean: {csv_file_path_clean}")
                            print(f"dstn_file_path: {dstn_file_path}")
                        # Copy and delete CSV file to "lh_bronze/Processed/CSV" (archive) - End

                        # Insert "Success" in lh_log (for each file) - Start
                        # Always log "Success" for extracted object.
                        alert             = "Success"
                        alert_description = f"Row count: {str(sdf_csv_file_count)}"

                        if is_debug:
                            par["locals"] = locals()
                            fn_print_debug_info(alert, fn_name, par)
                            del par

                        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)
                        # Insert "Success" in lh_log (for each file) - End
                        
                        if is_debug: print("--------------- Loop - End ---------------")

            alert             = "Success"
            alert_description = f""

            return (alert, alert_description)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            if is_debug:
                par["locals"] = locals()
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

            return (alert, alert_description)

In [ ]:
if is_debug: print("\n---------- This notenook name: nb_bronze_function_extract_csv - End ----------\n")